### CS2101 - Programming for Science and Finance
Prof. Götz Pfeiffer<br />
School of Mathematical and Statistical Sciences<br />
University of Galway

***

# Week 11: Function Parameters,  Decorators

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import math

## Decorating a Function

* Suppose that, for a  certain function `f`, we would like to have a message printed that tells us when this function is called.
* For example, the function `factorial` that computes the factorial $n!$ of a natural number `n`.
* Recall the recursive formula
  $$
  n! = \begin{cases}
  1, & n = 0 \\
  (n-1)! \cdot n, & \text{else}
  \end{cases}
  $$

In [ ]:
def factorial(n):
    return 1 if n == 0 else factorial(n-1) * n

In [ ]:
factorial(10)

* We could simply add a `print` statement to the body of the function.

In [ ]:
def factorial_m(n):
    print("calling factorial function")
    return 1 if n == 0 else factorial_m(n-1) * n

In [ ]:
factorial_m(5)

* Works, but feels quite intrusive.
* Perhaps we want to do this to more than just one function.  And perhaps we want to get rid of these messages when we get tired of them ...
* Wouldn't it be nice if we could write a **function** `add_message` that does this:  take any function as an argument, then construct and return that function with the added message whenever it gets called!
* Here we go.

In [ ]:
def add_message(f):
    def f_m(x):
        print(f'calling {f.__name__} function')
        return f(x)
    return f_m

* Test drive with `factorial`.

In [ ]:
factorial_mm = add_message(factorial)
factorial_mm(5)

In [ ]:
factorial_mm.__name__

* Not quite right ...

In [ ]:
factorial = add_message(factorial)
factorial(4)

* better ...

### Decorators

* Python provide a syntactic shorthand for situations like this.
* Instead of
  ```python
     def factorial(n):
         return ...
     factorial = add_message(factorial)
  ```
  we can write
  ```python
     @add_message
     def factorial(n):
         return ...
  ```
  for the same effect.
* The `@add_message` part is called a **decorator**.
* In general, a decorator allows you to execute additional code before or after a function call.

In [ ]:
@add_message
def factorial(n):
    return 1 if n == 0 else factorial(n-1) * n

In [ ]:
factorial(5)

* Let's add a message before and after the function call.

In [ ]:
def add_messages(f):
    def f_m(x):
        print(f'calling {f.__name__} function')
        value = f(x)
        print(f'returns {value}')
        return value
    return f_m

In [ ]:
@add_messages
def factorial(n):
    return 1 if n == 0 else factorial(n-1) * n

In [ ]:
factorial(4)

* In principle this should work for functions with more than one argument as well.
* However, if we define for example
  ```python
     @add_messages
     def gcd(a, b):
         return a if b == 0 else gcd(b, a % b)
  ```
  then the function call
  ```python
     gcd(60, 24)
  ```
  will fail with an error message, tellung us that the function `f_m` inside `add_messages` expects 1 argument, not 2.

* Perhaps we need a separate decorator for the 2 argument case ...

In [ ]:
def add_messages2(f):
    def f_m(x, y):
        print(f'calling {f.__name__} function')
        value = f(x, y)
        print(f'returns {value}')
        return value
    return f_m

In [ ]:
@add_messages2
def gcd(a, b):
    return a if b == 0 else gcd(b, a % b)

In [ ]:
gcd(60, 24)

* Works, but feels a bit repetitive.
* What if the function we'd like to decorate has 3 arguments, or 4 ...?
* A better solution uses a multiple arguments parameter `*args`.

In [ ]:
def add_messages_x(f):
    def f_m(*args):
        print(f'calling {f.__name__} function')
        value = f(*args)
        print(f'returns {value}')
        return value
    return f_m

* Now we can decorate functions regardless of their number of positional arguments.

In [ ]:
@add_messages_x
def factorial(n):
    return 1 if n == 0 else factorial(n-1) * n

In [ ]:
factorial(3)

In [ ]:
#@add_messages_x
def gcd(a, b):
    return a if b == 0 else gcd(b, a % b)

In [ ]:
gcd(60, 24)

* But what about keyword arguments?

##  Positional vs. Keyword arguments

* In a Python **function definition**, a parameter can be given a **default value**.
* A function parameter with a default value is an **optional parameter**, and one without is a **required parameter**.
* This means that when the function is called, for each required parameter there **must be** an argument value in the function call, for each optional parameter there **can be** an argument.

In [ ]:
def f(a, b, c=0, d=1): print(a, b, c, d)  # silly example

In [ ]:
f(1,2,3)

<div class="alert alert-danger">
    
* **Rule:** required parameters come before optional parameters.
</div>

* In a Python **function call**, there are **positional arguments** and **keyword arguments**.
* Positional arguments are matched up with parameters according to (guess what) their **position**.
* Keyword arguments are matched up by **name**.

In [ ]:
f(1, d=2, b=3, c=4)

* Despite their looks, positional arguments need not correspond to required parameters.
* In principle, all parameters can be assigned positional arguments.
* In principle, all parameters can be assigned keyword arguments.
* However, if one of the parameters is `*args` there are some restrictions, see below.

In [ ]:
f(5, 6, 7, 8)

In [ ]:
f(b='b', a='a', d='d', c='c')

<div class="alert alert-danger">
    
* **Rule:** positional arguments come before keyword arguments.
</div>

* If `*args` is one of the parameters in the function definition, then a function call can have **more positional arguments** than there are declared parameters.  Any additional positional arguments will then be collected in tuple `arg`.
* If  `**kwargs` is one of the parameters in the function definition, then a function can have **more keyword arguments** than there are declared parameters.  Any additional keyword arguments will then be collected in a dictionary `kwargs`.
* Here, `args` and `kwargs` are just arbitrary names, what matters are the stars `*` and `**`.

<div class="alert alert-danger">
    
* **Rule:** `**kwargs` must be the last of of the parameters.
</div>

In [ ]:
def f(a, b, *args, c, d, **kwargs):
    print(a, b, args, c, d, kwargs)

In [ ]:
f(1, 2, 3, c=4, d=5, e=6)

<div class="alert alert-danger">
    
* **Rule:** All the parameters before `*args` can only get positional arguments, and all the parameters after `*args` can only get keyword arguments.
</div>

* For `*args` to work, all the parameters before `*args` can only get positional arguments, and all the parameters after `*args` can only get keyword arguments.
* If `*args` is not an argument, Python allows us to explicitly introduce such restrictions with the virtual parameters `/` and `*`.
* A parameter `/` marks the end of the parameters that are positional only.
* A parameter `*` marks the start of the parameters that are keyword only.

In [ ]:
def f(a, b, /, c, d, *, e, f):
    print(a, b, c, d, e, f)

* Here `a` and `b` are **positional only** parameters, `c` and `d` are **standard parameters** (meaning they can get positional or keyword arguments), `e` and `f` are **keyword only** parameters.
* A valid call to this function looks like:

In [ ]:
f(1, 2, 3, d=4, e=5, f=6)

* In any case, the arguments of any function call to a function `f` are **positional arguments followed by keyword arguments**, always matching the pattern `f(*args, **kwargs)`.
* So if we define a function with signature `ff(*args, **kwargs)`, we can pass on its arguments, whatever they are, to a function call `f(*args, **kwargs)` in its body.
* It seems appropriate to use this technique in a decorator.

In [ ]:
def add_messages_y(f):
    def f_m(*args, **kwargs):
        print(f'calling {f.__name__} function with args {args} and kwargs {kwargs}')
        value = f(*args, **kwargs)
        print(f'returns {value}')
        return value
    return f_m

* For example, a general adder ...

In [ ]:
@add_messages_y
def my_sum(*args, zero=0):
    result = zero
    for a in args:
        result += a
    return result

In [ ]:
my_sum(1, 2, 3, 4)

In [ ]:
my_sum('a', 'b', 'c', zero="")

In [ ]:
my_sum([1], [2], [3,4], zero=[])

##  Class Based Decorators.

* A decorator need not be a function, it only has to be **callable**.
* An object `o` of a class `C` is callable, if the class `C` implements the special method `__call__`.
* Then a function call like `o(arg)` will refer back to `o.__call__(arg)`.
* We can thus write a **decorator class** whose objects become the decorated functions.
* This allows a proper separation of creating the decorated function, and its application in a function call.
  

* For example the logging decorator.

In [ ]:
class logger:
    def __init__(self, func):
        self.func = func
    def __call__(self, *args, **kwargs):
        print(f'calling {self.func.__name__} function with args {args} and kwargs {kwargs}')
        value = self.func(*args, **kwargs)
        print(f'returns {value}')
        return value


In [ ]:
@logger
def gcd(a, b):
    return a if b == 0 else gcd(b, a % b)

In [ ]:
gcd(60, 24)

* Recall the [Fibonacci sequence](https://en.wikipedia.org/wiki/Fibonacci_sequence) $F_n$ defined recursively as
  $$
  F_n = \begin{cases}
  n, & \text{if } 0 \leq n < 2,\\
  F_{n-2} + F_{n-1}, & \text{else.}
  \end{cases}
  $$
*  When implemented in this form as a function `fibonacci` it tends to need a lot of recursive calls to `fibonacci`, as can be demonstrtated with the `logger`. 

In [ ]:
@logger
def fibonacci(n):
    return n if n < 2 else fibonacci(n-2) + fibonacci(n-1)

In [ ]:
fibonacci(5)

### Memoizing Return Values

* Often, when a function gets called frequently with the same arguments, it pays off to store previously computed values and return them again, rather that recompute.

In [ ]:
class memoizer:
    def __init__(self, func):
        self.func = func
        self.cache = {}

    def __call__(self, *args, **kwargs):
        if args in self.cache:
            print(f"re-using cache value {self.cache[args]} for args {args}!")
            return self.cache[args]
        else:
            print(f'calling {self.func.__name__} function with args {args} and kwargs {kwargs}')
            value = self.func(*args, **kwargs)
            print(f'returns {value}')
            self.cache[args] = value
            return value
           

In [ ]:
@memoizer
def fibonacci(n):
    return n if n < 2 else fibonacci(n-2) + fibonacci(n-1)

In [ ]:
fibonacci(5)

In [ ]:
fibonacci(10)

## References

### Python

* [Decorators](https://book.pythontips.com/en/latest/decorators.html)

### Numpy


### Other


##  Exercises

1.  Write a decorator function `counted` that, when applied to a function definition for a function `func`, keeps track of the number of times `func` is called in the attribute `count` of `func`.

In [ ]:
def counted(func):
    def counted_func(*args, **kwargs):
        counted_func.count += 1
        return func(*args, **kwargs)
    counted_func.count = 0
    return counted_func

2.  Apply the `@counted` decorator to your favorite implementation `fibonacci` of the Fibonacci numbers.  Compute `fibonacci(20)` and check the value of the attribute `fibonacci.count`.

In [ ]:
@counted
def fibonacci(n):
    return n if n < 2 else fibonacci(n-1) + fibonacci(n-2)

In [ ]:
print(fibonacci(20))
fibonacci.count

3.  Write a decorator function `timed` that, when applied to a function definition for a function `func` prints a message containing the time spent with any call of `func`.

In [ ]:
from time import perf_counter

def timed(func):
    def timed_func(*args, **kwargs):
        start = perf_counter()
        value = func(*args, **kwargs)
        stop = perf_counter()
        runtime = stop -  start
        print(f'{func.__name__}: {runtime:4f} sec')
        return value
    return timed_func
    

4. Apply the `@timed` decorator to your favorite implementation of `gcd` and then compute the gcd of two large numbers.

In [ ]:
@timed
def gcd(a, b):
    return a if b == 0 else gcd(b, a % b)

In [ ]:
gcd(60354678907625, 24345678909875)

5.  Write a decorator **class** `timer` that, when applied to a function definition for a function `func` prints a message containing the time spent with any call of `func`.  Apply this decorator to your favorite implemntation of `gcd`, then compute the gcd of two large numbers.

In [ ]:
class timer:
    def __init__(self, func):
        self.func = func
    def __call__(self, *args, **kwargs):
        clock = -perf_counter()
        value = self.func(*args, **kwargs)
        clock += perf_counter()
        print(f'{self.func.__name__}: {clock:4f} sec')
        return value

In [ ]:
@timer
def gcd(a, b):
    return a if b == 0 else gcd(b, a % b)

In [ ]:
gcd(60354678907625, 24345678909875)